In [1]:
import numpy as np
import pandas as pd

In [4]:
data_total = pd.read_csv('./ESL-data/ESLmixture.csv').iloc[:,1:]
data_total.head(n=5)

,X1,X2,label
0,2.526093,0.321050,0
1,0.366954,0.031462,0
2,0.768219,0.717486,0
3,0.693436,0.777194,0
4,-0.019837,0.867254,0


In [13]:
train_data = data_total.iloc[:,[0,1]]
label = data_total["label"][:,np.newaxis]

In [14]:
num_data, num_feature = train_data.shape

In [16]:
# Initialize some paramters
alpha_vec = np.zeros((num_data,1))
iter_num = 50
C = 100

In [20]:
def find_new_index(i):
    j = np.random.randint(1, iter_num+1, 1)
    if (i == j):
        return(find_new_index)
    else:
        return(j)

In [26]:
# main function for svm-smo
iter_i = 0
intercept = 0
while iter_i <= iter_num:
    alpha_changed = 0
    for i in range(num_data):
        gX_i = np.inner(alpha_vec, label).T * (np.dot(train_data, train_data[i,:].T)) + intercept
        E_i = gX_i - label[i]
        criterion = label[i] * gX_i
        if (((label[i]*E_i < -1e3) and (alpha_vec[i] < C)) 
            or ((label[i]*E_i > 1e3) and (alpha_vec[i] > 0))):
            j = find_new_index(i)
            gX_j = (alpha_vec * label).T * (np.dot(train_data, train_data[j,:].T)) + intercept
            E_j = gX_j - label[j]
            alpha_old_i = alpha_vec[i]
            alpha_old_j = alpha_vec[j]
            
            if label[i] != label[j]:
                L = np.max(0, alpha_old_j - alpha_old_i)
                H = np.min(C, C + alpha_old_j - alpha_old_i)
            else:
                L = np.max(0, alpha_old_j + alpha_old_i - C)
                H = np.min(C, alpha_old_j + alpha_old_i)
            
            if L==H:
                continue
            
            x1 = train_data[i,:]
            x2 = train_data[j,:]
            eta = 2 * np.inner(x1, x2) - np.inner(x1,x1) - np.inner(x2,x2)
            alpha_new_j = alpha_old_j - label[j] * (E_i - E_j) / eta
            
            if alpha_new_j > H:
                alpha_new_j = H
            elif alpha_new_j < L:
                alpha_new_j = L
                
            if np.abs(alpha_new_j - alpha_old_j) < 1e-4:
                continue
            else:
                alpha_new_i = alpha_old_j + label[i]*label[j]*(alpha_old_j - alpha_new_j)
            
            # intercept decision
            intercept_new_1 = (intercept - E_i - label[i]*(alpha_new_i - alpha_old_i)*np.inner(x1,x1) 
                               - label[j]*(alpha_new_j - alpha_old_j)*np.inner(x1,x2))
            intercept_new_2 = (intercept - E_j - label[i]*(alpha_new_i - alpha_old_i)*np.inner(x1,x2) 
                               - label[j]*(alpha_new_j - alpha_old_j)*np.inner(x2,x2))
            
            if (alpha_new_i > 0) and (alpha_new_i < C):
                intercept = intercept_new_1
            elif (alpha_new_j > 0) and (alpha_new_j < C):
                intercept = intercept_new_2
            else:
                intercept = (intercept_new_1 + intercept_new_2) / 2
            alpha_vec[i] = alpha_new_i
            alpha_vec[j] = alpha_new_j
            alpha_changed += 1
            
    if alpha_chenged == 0:
        iter += 1
    else:
        iter = 0
        
                

SyntaxError: invalid syntax (<ipython-input-26-b06052d7cf39>, line 49)